# 00 - Foundry IQ Exploration

## Content
1. Create a Project Connection: to authorize the MCP Tool
2. Create the Agent with a MCP Tool
3. Invoke the Agent and Query the data

In [49]:
import os
from dotenv import load_dotenv

In [50]:
load_dotenv(override=True)

True

In [ ]:
# Environment  variables
FOUNDRY_PROJECT_RESOUCE_ID = os.getenv("FOUNDRY_PROJECT_RESOUCE_ID")
FOUNDRY_PROJECT_ENDPOINT = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
KNOWLEGE_BASE_NAME = os.getenv("KNOWLEGE_BASE_NAME")

MCP_ENDPOINT = f"{AI_SEARCH_ENDPOINT}/knowledgebases/{KNOWLEGE_BASE_NAME}/mcp?api-version=2025-11-01-preview"
FOUNDRY_PROJECT_CONNECTION_NAME = os.getenv("FOUNDRY_PROJECT_CONNECTION_NAME")

AGENT_NAME = os.getenv("AGENT_NAME")
AGENT_MODEL = os.getenv("AGENT_MODEL")

### 1 - Create a Project Connection
Can be checked on Microsoft Foundry (New) > Build > Tools

In [52]:
import requests
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Get Bearer Token
bearer_token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://management.azure.com/.default")
headers = {
  "Authorization": f"Bearer {bearer_token_provider()}",
}

In [55]:
# Create Project Connection
response = requests.put(
  f"https://management.azure.com{FOUNDRY_PROJECT_RESOUCE_ID}/connections/{FOUNDRY_PROJECT_CONNECTION_NAME}?api-version=2025-10-01-preview",
  headers=headers,
  json = {
    "name": FOUNDRY_PROJECT_CONNECTION_NAME,
    "type": "Microsoft.MachineLearningServices/workspaces/connections",
    "properties": {
      "authType": "ProjectManagedIdentity",
      "category": "RemoteTool",
      "target": MCP_ENDPOINT,
      "isSharedToAllUsers": True,
      "audience": "https://search.azure.com/",
      "metadata": { "ApiType": "Azure" }
    }
  }
)

response.raise_for_status()
print("Project Connection created successfully.")

Project Connection created successfully.


### 2 - Create the Agent with a MCP Tool

In [56]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, MCPTool

# Create Foundry Project Client
project_client = AIProjectClient(
  endpoint=FOUNDRY_PROJECT_ENDPOINT,
  credential=DefaultAzureCredential()
)

In [57]:
# Agent Instructions
instructions = """
You are a helpful assistant that must use the knowledge base to answer all the questions from user. You must never answer from your own knowledge under any circumstances.
Every answer must always provide annotations for using the MCP knowledge base tool and render them as: `【message_idx:search_idx†source_name】`
If you cannot find the answer in the provided knowledge base you must respond with "I don't know".
"""

In [58]:
# Create the MCP Tool
mcp_kb_tool = MCPTool(
  server_label="knwoledge-base",
  server_url=MCP_ENDPOINT,
  require_approval="never",
  allowed_tools=["knowledge_base_retrieve"],
  project_connection_id=FOUNDRY_PROJECT_CONNECTION_NAME
)

In [59]:
# Create the Agent
agent = project_client.agents.create_version(
  agent_name=AGENT_NAME,
  definition=PromptAgentDefinition(
    model=AGENT_MODEL,
    instructions=instructions,
    tools=[mcp_kb_tool]
  )
)

print(f"Agent '{AGENT_NAME}' created successfully with version: {agent.version}.")

Agent 'foundry-iq-agent-1229' created successfully with version: 1.


In [60]:
import json
print(json.dumps(agent.as_dict(), indent=2))

{
  "metadata": {},
  "object": "agent.version",
  "id": "foundry-iq-agent-1229:1",
  "name": "foundry-iq-agent-1229",
  "version": "1",
  "description": "",
  "created_at": 1766978091,
  "definition": {
    "kind": "prompt",
    "model": "gpt-4o",
    "instructions": "\nYou are a helpful assistant that must use the knowledge base to answer all the questions from user. You must never answer from your own knowledge under any circumstances.\nEvery answer must always provide annotations for using the MCP knowledge base tool and render them as: `\u3010message_idx:search_idx\u2020source_name\u3011`\nIf you cannot find the answer in the provided knowledge base you must respond with \"I don't know\".\n",
    "tools": [
      {
        "type": "mcp",
        "server_label": "knwoledge-base",
        "server_url": "https://ai-search-iq.search.windows.net/knowledgebases/iq-knowledge-base-1226/mcp?api-version=2025-11-01-preview",
        "allowed_tools": [
          "knowledge_base_retrieve"
    

### 3 - Invoke the Agent and Query the data

In [64]:
import openai

# Get OpenAI Client from Foundry Project Client
openai_client = project_client.get_openai_client()

conversation = openai_client.conversations.create()

# Query the Agent
response = openai_client.responses.create(
  conversation=conversation.id,
  tool_choice="required",
  input="""
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
  """,
  extra_body={
    "agent": {
      "name": agent.name,
      "type": "agent_reference"
    }
  }
)

In [66]:
print(response.output_text)

1. Suburban belts experience larger December brightening compared to urban cores due to the widespread use of holiday lighting in single-family homes, which are more common in suburban areas. This increases light levels significantly during the holiday season, whereas urban cores see less of an increase due to fewer private homes and greater limitations on holiday displays.

2. In Phoenix, the sharply visible nighttime street grid is due to its urban design, which follows a regular grid structure typical of cities in the western U.S. This is complemented by extensive street lighting across its metropolitan expanse. In contrast, interstate stretches between midwestern cities are dim, as they are often surrounded by sparsely populated rural areas with minimal lighting, reflecting the dominance of urban nodes over continuous light networks.
